In [ ]:
# Author: Andrew Louwagie Gordon
# Date Created: 22May2018
# Last Modified: 01Jun2018

In [ ]:
# Import Block
# Import the necessary packages
import ipywidgets as widgets
import bqplot as bq
import numpy as np
import matplotlib.pyplot as plt
import tempNcolor as tc
import expToLaTeX as e2l

In [ ]:
# Function Definitions Block

def newRad(r):
    '''
    This function resizes the star in the figure to be the correct size based on slider value.
    '''
    radius = 10 * r
    return radius

def Star_Temp(T):
    '''
    This function calculates the temperature of the star in Kelvin.
    '''
    temp = T * T_Sun
    temp = round(temp, -2) # Round the temperature to the nearest 100 K
    return temp

def L_Ratio(t, r):
    '''
    This function calculates the ratio of luminosities for the star based on temperature and radius.
    '''
    lum = (r ** 2.0) * (t ** 4.0) # Luminosity calculation in L/L_sun
    # The large set of 'if' statements following this line rounds the result to two significant figures.
    if lum < 0.01:  
        lum = round(lum, 4)
    elif lum >= 0.01 and lum < 0.1:
        lum = round(lum, 3)
    elif lum >= 0.1 and lum < 1:
        lum = round(lum, 2)   
    elif lum >= 1 and lum < 10:
        lum = round(lum, 1)    
    elif lum >= 10 and lum < 100:
        lum = round(lum, 0)    
    elif lum >= 100 and lum < 1000:
        lum = round(lum, -1)
    elif lum >= 1000 and lum < 10000:
        lum = round(lum, -2)
    elif lum >= 10000 and lum < 100000:
        lum = round(lum, -3)
    else:
        lum = round(lum, -4)
    return lum

def cr(change=None): # cr stands for color and radius
    '''
    This function continuously updates the color and radius of the star.
    '''
    x1 = 2
    x2 = 7
    t_star = Star_Temp(Temp.value)
    counts = tc.temp2rgb(t_star)
    hex_color = tc.rgb2hex(counts)
    rad_new = 100 * int(newRad(Rad.value))
    star2.colors = [hex_color[0]]
    star2.default_size = rad_new

def w(change=None): # w stands for widgets
    '''
    This function continuously updates the widgets that display information.
    '''
    get_l_ratio = L_Ratio(Temp.value, Rad.value)
    L_Ratio_report.value = str(get_l_ratio)
    Luminosity = get_l_ratio * L_Sun
    list = e2l.exp2LaTeX(Luminosity)
    Luminosity_report.value = r'\({}\)'.format(list[1])
    t_star = Star_Temp(Temp.value)
    t_star_report.value = str(t_star)

In [ ]:
# Variable Definitions Block
L_Sun = 3.8e26 # Solar luminosity in Watts
T_Sun = 5777 # Solar temperature in Kelvin
t_star = T_Sun # Define variable to be updated later
Luminosity = 1 # Define variable to be updated later
get_l_ratio = 1 # Define variable to be updated later
list = e2l.exp2LaTeX(Luminosity) # Make a list from the number2LaTeX converter being used

In [ ]:
# Widgets Definitions Block
# Radius slider in units of R/R_Sun
Rad = widgets.FloatSlider(
    min=0.2, 
    max=13.4, 
    step=0.1, 
    disabled=False, 
    continuous_update=True, 
    orientation='horizontal', 
    readout=True, 
    readout_format='.1f',
)

# Temperature slider in units of T/T_Sun
Temp = widgets.FloatSlider(
    min=0.5,
    max=7.0,
    step=0.1,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Widget to report updated temperature in Kelvin
t_star_report = widgets.Label(
    value = str(int(t_star)),
    placeholder = 'Type something',
    disabled = True   
)

# Widget to report updated luminosity in L/L_sun
L_Ratio_report = widgets.Label(
    value = str(get_l_ratio),
    placeholder = 'Type something',
    disabled = True   
)

# Widget to report updated luminosity in Watts 
Luminosity_report = widgets.Label(
    value = r'\({}\)'.format(list[1]),
    placeholder = 'Type something',
    disabled = True   
)

In [ ]:
# Calculations Block
t_star = Star_Temp(Temp.value) # Temperature calculation
get_l_ratio = L_Ratio(Temp.value, Rad.value) # Luminosity ratio calculation
Luminosity = get_l_ratio * L_Sun # Convert luminosity to Watts

In [ ]:
# Display Figures and Widgets Block
# These are the parameters for the visual plot
x_arr = np.linspace(1,10,num=100)
y_arr = x_arr
sc_x = bq.LinearScale()
sc_y = bq.LinearScale()

# These are the stars coordinates in the figure
x1 = 2
x2 = 7

t_star = Star_Temp(Temp.value) # Get the initial temperature for the star
counts = tc.temp2rgb(t_star) # Convert temperature into rgb counts
hex_color = tc.rgb2hex(counts) # Convert the rgb counts into a hexidecimal value that reflects the color of the star
rad_new = newRad(Rad.value) # Get the radius of the star

# Labels the stars on the plot
label = bq.Label(x=(1.5,5.75), y=[6,1.5], scales={'x': sc_x, 'y': sc_y},
                   text=['Sun', 'Model Star'], default_size=15, font_weight='bolder',
                   colors=['white'], update_on_move=False)

# Define the initial properties of the stars for the plot
star1 = bq.Scatter(x=[x1], y=[5], scales={'x': sc_x, 'y': sc_y}, names = ['Sun'], colors = ['lightgoldenrodyellow'], 
                   default_size = 1000)
star2 = bq.Scatter(x=[x2], y=[5], scales={'x': sc_x, 'y': sc_y}, names = ['Hypothetical Star'], colors = [hex_color[0]], 
                   default_size = 100 * int(rad_new))

# This holds the plot so that the stars are centered between the top to bottom, I have no idea why it works because it is not 
#      called or referenced by the figure later, but it does affect the figure in a negative way when removed.  Therefore, this 
#      statement stays until I can figure out what is going on.
lin = bq.Scatter(x=x_arr, y=y_arr, scales={'x': sc_x, 'y': sc_y}, opacites=[0])

# Use the cr function to continuously update star 2 in the plot
Temp.observe(cr, names=['value'])
Rad.observe(cr, names=['value'])

# Use the w function to continuously update the calculated values in the display widgets to the right
Temp.observe(w, names=['value'])
Rad.observe(w, names=['value'])

# Define the parameters for the axes so the plot appears presentable
ax_x = bq.Axis(scale=sc_x, grid_color = 'black', num_ticks = 0)
ax_y = bq.Axis(scale=sc_y, grid_color = 'black', num_ticks = 0, tick_format='0.2f', orientation='vertical')

# Generate the plot
fig = bq.Figure(title = 'Modeling a Star', marks=[star1, star2, label], axes=[ax_x, ax_y], animation = 1000, min_aspect_ratio = 1, max_aspect_ratio = 1,
                background_style ={'fill':'black'})

# Define the layout for the final widget to make it presentable
box_layout = widgets.Layout(align_items='center', justify_content = 'flex-end', border='none', width='100%')

# Organize all of the widgets in a coherent way and display them
top = fig
rad_slide = widgets.HBox([widgets.Label ('Radius (R/R_Solar):'),Rad])
temp_slide = widgets.HBox([widgets.Label('Temperature (T/T_Solar):'),Temp])
temp_disp = widgets.HBox([widgets.Label('Temperature (K):'), t_star_report])
lratio_disp = widgets.HBox([widgets.Label('Luminosity (L/L_Solar):'), L_Ratio_report])
lum_disp = widgets.HBox([widgets.Label('Luminosity (W):'), Luminosity_report])
lum_disp.children[1].layout.width = '100px' # Make the widget large enough to handle the LaTeX notation 
bottom_left = widgets.VBox([temp_slide, rad_slide])
bottom_right = widgets.VBox([temp_disp, lratio_disp, lum_disp])
bottom = widgets.HBox([bottom_left, bottom_right])
box = widgets.VBox([top, bottom], layout = box_layout)
box

In [ ]:
# Note to Author: Possible add-ins
'''
-No notes or updates at this time
'''